In [ ]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv("../../data/combine_df.csv")

In [ ]:
df.head()

In [ ]:
# Function to create NER datasets
def create_ner_datasets(df):
    for service in df['service'].unique():
        service_df = df[df['service'] == service]
        ner_data = []
        
        for _, row in service_df.iterrows():
            # Replace single quotes with double quotes
            entities_str = row['entities'].replace("'", '"')
            
            try:
                entities = json.loads(entities_str)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue  # Skip this row if JSON is invalid
                
            tokens = []
            
            # Extract entities for labeling
            for key, value in entities.items():
                tokens.append((value, key))  # (text, label)
                
            ner_data.append(tokens)

        # Save to file named after the service
        with open(f"../../data/ner_dataset/{service}_ner_dataset.txt", "w") as file:
            for entry in ner_data:
                file.write(f"{entry}\n")

In [ ]:
# Create NER datasets
create_ner_datasets(df)

In [ ]:
import json

# Sample data from your dataset
data = [
    [('required for my job responsibilities', 'purpose_of_access')],
]

def convert_to_bio_format(data):
    bio_data = []
    
    for entry in data:
        tokens = []
        labels = []
        
        for text, label in entry:
            # Split the text into tokens
            token_list = text.split()  # Basic tokenization
            
            # Create BIO labels
            for i, token in enumerate(token_list):
                if i == 0:
                    labels.append(f'B-{label}')  # Beginning of an entity
                else:
                    labels.append(f'I-{label}')  # Inside an entity
            
            tokens.extend(token_list)

        # Combine tokens and labels
        bio_entry = list(zip(tokens, labels))
        bio_data.append(bio_entry)

    return bio_data

# Convert the data
bio_formatted_data = convert_to_bio_format(data)

# Save to a file or print
with open('ner_training_data.json', 'w') as f:
    json.dump(bio_formatted_data, f, indent=2)

# Example output
for entry in bio_formatted_data:
    print(entry)